In [1]:
# 공통 환경 설정
from dotenv import load_dotenv
import os

# 작업디렉토리를 상위경로로 변경
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
os.chdir(parent_dir)

# 환경변수 설정
load_dotenv(dotenv_path=".env.testcase", override=True)

from app.db_model.database import SessionLocal
from app.vectordb.faiss_vectordb import FaissVectorDB


session = SessionLocal()
faissVectorDB = FaissVectorDB(db_session=session, index_name='cg_code_assist')

2025-01-21 07:47:21,461 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-01-21 07:47:21,462 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-21 07:47:21,464 INFO sqlalchemy.engine.Engine select current_schema()
2025-01-21 07:47:21,465 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-21 07:47:21,466 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-01-21 07:47:21,467 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-01-21 07:47:21,468 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-21 07:47:21,482 INFO sqlalchemy.engine.Engine SELECT faiss_info.id, faiss_info.index_name, faiss_info.index_desc, faiss_info.index_file_path, faiss_info.modified_at, faiss_info.created_at, faiss_info.modified_by, faiss_info.created_by 
FROM faiss_info 
WHERE faiss_info.index_name = %(index_name_1)s
2025-01-21 07:47:21,483 INFO sqlalchemy.engine.Engine [generated in 0.00071s] {'index_name_1': 'cg_code_assist'}
2025-01-21 07:47:21,493 INFO sqlalchemy.engine.Engine SELECT

In [ ]:
# 예시 쿼리와 타겟 리스트
from app.process.reranker import AlfredReranker

query = "public Optional<SensorDataDto> getPigVital(Long vitalCode, Long pigCode)"

# flash_rank_rerank 함수 호출
reranker = AlfredReranker()
results = reranker.cross_encoder(query, faissVectorDB, k=5)

# 결과 출력
for result in results:
    print(f"Chunk: {result}")

In [ ]:
from app.chain_graph.agent_state import CodeAssistState
from app.chain_graph.code_assist_chain import CodeAssistChain
from app.db_model.database import SessionLocal
from app.vectordb.faiss_vectordb import FaissVectorDB
# session = SessionLocal()
# faissVectorDB = FaissVectorDB(db_session=session, index_name='cg_code_assist')    
    
state = CodeAssistState()
state["question"] = "스페인의 비는 어디에 내리나요?"

code_assist_chain = CodeAssistChain()
code_assist_state = code_assist_chain.contextual_reranker(state=state, k=3)

In [2]:
# 공통 환경 설정
from dotenv import load_dotenv
import os

# 작업디렉토리를 상위경로로 변경
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
os.chdir(parent_dir)

# 환경변수 설정
load_dotenv(dotenv_path=".env.testcase", override=True)

from app.db_model.database import SessionLocal
from app.vectordb.faiss_vectordb import FaissVectorDB

session = SessionLocal()
faissVectorDB = FaissVectorDB(db_session=session, index_name='cg_code_assist')


from fastapi.responses import StreamingResponse
from app.chain_graph.agent_state import CodeAssistChatState
from app.chain_graph.code_assist_chain import CodeAssistChain
state = CodeAssistChatState()
state['question'] = '스페인의 비는 어디에 내리나요?'
state['current_code'] = ''
state['chat_history_id'] = '1'
state['chat_history'] = []

code_assist_chain = CodeAssistChain()
response = await code_assist_chain.get_chain(task_type="chat").ainvoke(state)

# async def stream_response() :
#     async for chunk in code_assist_chain.get_chain(task_type="chat").astream(state, stream_mode="custom"):
#         print("## chucnk=", chunk.content)
#         yield chunk.content

# StreamingResponse(stream_response(), media_type="text/event-stream")



### cg_code_assist 인덱스를 메모리에서 로드했습니다.
### cg_code_assist 인덱스를 메모리에서 로드했습니다.
### Index already exists: cg_code_assist
